In [2]:
from ensemble_fns import (adaboost,
                          adaboost_pred,
                          adaboost_acc,
                          random_forest, 
                          bagging, 
                          forest_pred,
                          forest_acc)
import sys
sys.path.append('..')
from datasets import get_bank_data
from decision_tree_fns import predict, tree_maker, ID3
import os
import pickle
import numpy as np
import pandas as pd

In [3]:
os.listdir('../datasets/credit_card')

['default of credit card clients.xls']

In [6]:
! pip install xlrd ## for pd.read_excel()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01


In [7]:
dataset_fp = os.path.join('../datasets/credit_card',os.listdir('../datasets/credit_card')[0])
df = pd.read_excel(dataset_fp)

In [8]:
print('preview df', df)

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [24]:
dataset = df.to_numpy()
X = dataset[1:,1:-1] #all rows except first, all cols except first or last
y = dataset[1:,-1]   #all rows except first, only last column
feat_names = dataset[0,1:-1]  #first row is headers, include all columns except first/last

In [37]:
print(f'loaded credit card data from xls file {y.shape=} {X.shape}, {feat_names.shape=}')

loaded credit card data from xls file y.shape=(30000,) (30000, 23), feat_names.shape=(23,)


In [38]:
np.random.seed(seed=0)

shuffle = np.random.choice(len(X), 30000, replace=False)
split = 24000

# be careful that columns of X_train might not have all the feature values that X_test does
X,y = X[shuffle], y[shuffle]
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]


In [ ]:
print(f'split data, {X_train.shape=}, {y_train.shape=}, {X_test.shape=}, {y_test.shape=}')

### handle continuous features using median splitting 

In [41]:
# continuous features are limit bal [0], age [4], bill amt 1 [11], rest of features [12:]
# [] denotes the column
numeric_features = [0,4,11,12,13,14,15,16,17,18,19,20,21,22]
print('continous features are', feat_names[numeric_features])

medians = np.median(X_train[:, numeric_features], axis=0)
X_train[:,numeric_features] = X_train[:,numeric_features] > medians
X_test[:,numeric_features] = X_test[:,numeric_features] > medians

continous features are ['LIMIT_BAL' 'AGE' 'BILL_AMT1' 'BILL_AMT2' 'BILL_AMT3' 'BILL_AMT4'
 'BILL_AMT5' 'BILL_AMT6' 'PAY_AMT1' 'PAY_AMT2' 'PAY_AMT3' 'PAY_AMT4'
 'PAY_AMT5' 'PAY_AMT6']


In [ ]:
single_tree = ID3(X,y,feat_names, max_depth=10, IG_metric='entropy')

tree_train_acc = (y_train == [predict(X_train[ex],bank_tree,feat_names) for ex in range(len(X_train))]).mean()
tree_test_acc = (y_test == [predict(X_test[ex],bank_tree,feat_names) for ex in range(len(X_test))]).mean()

print(f'single tree accuracies, {tree_train_acc} train, {tree_test_acc} test')


In [ ]:
stumps, amount_of_says = adaboost(X,
                                  y,
                                  feat_names, 
                                  num_stumps=500, 
                                  IG_metric='entropy')


ada_train_acc = adaboost_acc(X_train,y_train,stumps,feat_names, amount_of_says)
ada_test_acc = adaboost_acc(X_test,y_test,stumps,feat_names, amount_of_says)

print(f'adaboost ensemble accuracies, {ada_train_acc} train, {ada_test_acc} test')

In [ ]:
random_forest = random_forest(X_train,
                              y_train,
                              feat_names,
                              tree_count=500,
                              max_depth=100, 
                              IG_metric='entropy')

rf_train_acc = forest_acc(X_train,y_train,random_forest,feat_names)
rf_test_acc = forest_acc(X_test,y_test,random_forest,feat_names)

print(f'random forest accuracies {rf_train_acc} train, {rf_test_acc} test')

In [ ]:
bagged_forest = bagging(X_train,
                        y_train,
                        feat_names,
                        tree_count=500,
                        max_depth=100,
                        IG_metric='entropy')

bagged_train_acc = forest_acc(X_train,y_train,bagged_forest,feat_names)
bagged_test_acc = forest_acc(X_test,y_test,bagged_forest,feat_names)

print(f'bagged forest accuracies {bagged_train_acc} train, {bagged_test_acc} test')